In [1]:
print("Ok")

Ok


In [2]:
%pwd

'd:\\MediChatbot-with-Generative-AI\\research'

In [3]:
import os
os.chdir("../")

In [4]:
%pwd

'd:\\MediChatbot-with-Generative-AI'

In [2]:
import langchain
print("LangChain version:", langchain.__version__)


LangChain version: 0.3.25


In [5]:
from langchain.document_loaders import PyPDFLoader, DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [6]:
#Extract Data From the PDF File
def load_pdf_file(data):
    loader= DirectoryLoader(data,
                            glob="*.pdf",
                            loader_cls=PyPDFLoader)
    documents=loader.load()
    return documents

In [10]:
loader = DirectoryLoader("Data/")


In [11]:
loader = DirectoryLoader("D:/MediChatbot-with-Generative-AI/Data/")


In [13]:
import os

# Create the directory if it doesn't exist
if not os.path.exists("Data"):
    os.makedirs("Data")


In [15]:
extracted_data=load_pdf_file(data='Data/')

In [16]:
#Split the Data into Text Chunks
def text_split(extracted_data):
    text_splitter=RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=20)
    text_chunks=text_splitter.split_documents(extracted_data)
    return text_chunks

In [17]:
text_chunks=text_split(extracted_data)
print("Length of Text Chunks", len(text_chunks))

Length of Text Chunks 5860


In [19]:
from langchain.embeddings import HuggingFaceEmbeddings

In [30]:
#Download the Embeddings from Hugging Face
def download_hugging_face_embeddings():
    embeddings=HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2')
    return embeddings

In [31]:
from sentence_transformers import SentenceTransformer

model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')


In [32]:
embeddings = download_hugging_face_embeddings()

In [77]:
query_result = embeddings.embed_query("Hello world")
print("Length", len(query_result))

Length 384


In [76]:
from dotenv import load_dotenv
load_dotenv()

True

In [79]:
PINECONE_API_KEY=os.environ.get('PINECONE_API_KEY')

In [82]:
from pinecone.grpc import PineconeGRPC as Pinecone
from pinecone import ServerlessSpec
import os

pc = Pinecone(api_key=PINECONE_API_KEY)

index_name = "medichatbot"

pc.create_index(
    name=index_name,
    dimension=384,
    metric="cosine",
    spec=ServerlessSpec(
        cloud="aws",
        region="us-east-1"
    )
)

{
    "name": "medichatbot",
    "metric": "cosine",
    "host": "medichatbot-sphygug.svc.aped-4627-b74a.pinecone.io",
    "spec": {
        "serverless": {
            "cloud": "aws",
            "region": "us-east-1"
        }
    },
    "status": {
        "ready": true,
        "state": "Ready"
    },
    "vector_type": "dense",
    "dimension": 384,
    "deletion_protection": "disabled",
    "tags": null
}

In [78]:
import os
os.environ["PINECONE_API_KEY"] = PINECONE_API_KEY

In [4]:

from langchain_community.vectorstores import Pinecone

docsearch = Pinecone.from_documents(
    documents=text_chunks,
    index_name=index_name,
    embedding= embeddings,
)

NameError: name 'text_chunks' is not defined